In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from tqdm import tqdm

import torch
import torchvision
from torch.nn.utils import vector_to_parameters, parameters_to_vector

import nnj
from nnj.utils import convert_to_nnj
from pytorch_laplace.hessian.contrastive import ArccosHessianCalculator
from pytorch_laplace.laplace.diag import DiagLaplace

sys.path.append("../../")
from plotting_fun import plot_latent, plot_reconstruction_with_latent, plot_std, plot_fancy_latent, plot_attention, plot_training

torch.manual_seed(0)

In [3]:
!pip install pytorch-metric-learning
!pip install faiss-gpu

  Using cached faiss-gpu-1.7.1.post2.tar.gz (40 kB)
  Preparing metadata (setup.py) ... done
Discarding https://files.pythonhosted.org/packages/17/76/47d0cc8161f4bf988583a2839bb1e56baf09d6b80cfa472b9eba4d5f543b/faiss-gpu-1.7.1.post2.tar.gz (from https://pypi.org/simple/faiss-gpu/): Requested faiss-cpu from https://files.pythonhosted.org/packages/17/76/47d0cc8161f4bf988583a2839bb1e56baf09d6b80cfa472b9eba4d5f543b/faiss-gpu-1.7.1.post2.tar.gz has inconsistent name: expected 'faiss-gpu', but metadata has 'faiss-cpu'
  Using cached faiss-gpu-1.7.1.post1.tar.gz (41 kB)
  Preparing metadata (setup.py) ... done
Discarding https://files.pythonhosted.org/packages/39/8d/b62bc92c8dd4b2a99d4a06b8804280f6445748b6d698eabb037e111080c7/faiss-gpu-1.7.1.post1.tar.gz (from https://pypi.org/simple/faiss-gpu/): Requested faiss-cpu from https://files.pythonhosted.org/packages/39/8d/b62bc92c8dd4b2a99d4a06b8804280f6445748b6d698eabb037e111080c7/faiss-gpu-1.7.1.post1.tar.gz has inconsistent name: expected 'faiss

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
from torchvision import datasets, transforms

from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
def train(model, loss_func, mining_func, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = model(data)
        indices_tuple = mining_func(embeddings, labels)
        loss = loss_func(embeddings, labels, indices_tuple)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(
                "Epoch {} Iteration {}: Loss = {}, Number of mined triplets = {}".format(
                    epoch, batch_idx, loss, mining_func.num_triplets
                )
            )


### convenient function from pytorch-metric-learning ###
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)


### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    print("Computing accuracy")
    accuracies = accuracy_calculator.get_accuracy(
        test_embeddings, test_labels, train_embeddings, train_labels, False
    )
    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))


device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)

batch_size = 256

dataset1 = datasets.MNIST("../../../data", train=True, download=True, transform=transform)
dataset2 = datasets.MNIST("../../../data", train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(
    dataset1, batch_size=batch_size, shuffle=True
)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=batch_size)

### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
model = nnj.Sequential(
    nnj.Conv2d(1, 32, 3, 1),
    nnj.Tanh(),
    nnj.Conv2d(32, 64, 3, 1),
    nnj.Tanh(),
    nnj.MaxPool2d(2),
    nnj.Flatten(),
    nnj.Linear(9216, 128),
    nnj.L2Norm(),
)
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
num_epochs = 1


### pytorch-metric-learning stuff ###
distance = distances.LpDistance()
loss_func = losses.ContrastiveLoss(
    pos_margin=0, neg_margin=0.8, distance=distance
)

mining_func = miners.TripletMarginMiner(
    margin=0.2, distance=distance, type_of_triplets="semihard"
)
accuracy_calculator = AccuracyCalculator(include=("precision_at_1",), k=1)
### pytorch-metric-learning stuff ###


for epoch in range(1, num_epochs + 1):
    train(model, loss_func, mining_func, device, train_loader, optimizer, epoch)
    test(dataset1, dataset2, model, accuracy_calculator)

Epoch 1 Iteration 0: Loss = 0.9724591374397278, Number of mined triplets = 560992
Epoch 1 Iteration 20: Loss = 0.8178108334541321, Number of mined triplets = 147869
Epoch 1 Iteration 40: Loss = 0.8217043280601501, Number of mined triplets = 156738
Epoch 1 Iteration 60: Loss = 0.8481734991073608, Number of mined triplets = 97370
Epoch 1 Iteration 80: Loss = 0.7788268327713013, Number of mined triplets = 77356
Epoch 1 Iteration 100: Loss = 0.8116377592086792, Number of mined triplets = 92069
Epoch 1 Iteration 120: Loss = 0.8057810664176941, Number of mined triplets = 86491
Epoch 1 Iteration 140: Loss = 0.794864296913147, Number of mined triplets = 55119
Epoch 1 Iteration 160: Loss = 0.7977604269981384, Number of mined triplets = 47115
Epoch 1 Iteration 180: Loss = 0.8576111197471619, Number of mined triplets = 96745
Epoch 1 Iteration 200: Loss = 0.7837868928909302, Number of mined triplets = 43934
Epoch 1 Iteration 220: Loss = 0.7627668976783752, Number of mined triplets = 37053


100%|██████████| 313/313 [00:03<00:00, 88.85it/s] 
/Users/warburg/miniconda3/envs/laplace/lib/python3.8/site-packages/faiss/contrib/torch_utils.py:51: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  x.storage().data_ptr() + x.storage_offset() * 4)


Computing accuracy
Test set accuracy (Precision@1) = 0.9822


# Laplace posthoc

In [ ]:
hessian_calculator = ArccosHessianCalculator(
    wrt="weight",
    shape="diagonal",
    speed="half",
    method="fix",
)

# if arccos, then remove normalization layer from model
model_nnj = convert_to_nnj(model[:-1])

laplace = DiagLaplace()
_prior_prec = 0.01 # weight of the l2 regularizer
_prior_prec_multiplier = 10000 #prior optimization (made a posteriori by the Laplace Redux guys)

In [6]:
max_pairs = 100

mean = parameters_to_vector(model.parameters())
prior_hessian = _prior_prec_multiplier *_prior_prec * torch.ones_like(mean)
hessian = 0
for batch in tqdm(train_loader):
    x, y = batch

    with torch.no_grad():
        embeddings = model(x)
        indices_tuple = mining_func(embeddings, y)

        # randomly choose 5000 pairs if more than 5000 pairs available.
        if len(indices_tuple[0]) > max_pairs:
            idx = torch.randperm(indices_tuple[0].size(0))[: max_pairs]
            indices_tuple = (
                indices_tuple[0][idx],
                indices_tuple[1][idx],
                indices_tuple[2][idx],
            )

        hessian += hessian_calculator.compute_hessian(x, model_nnj, indices_tuple)

precision = prior_hessian + hessian
std_deviation = 1.0 / precision.sqrt()

plot_attention(std_deviation)

  0%|          | 0/235 [00:00<?, ?it/s]

: 

: 

In [ ]:
indices_tuple

NameError: name 'indices_tuple' is not defined